In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bsp
from selenium import webdriver
import time
import re
import sqlite3

## 여성향수

In [2]:
def getData(whr=''):
    dbname = 'Oliveyoung.db'
    with sqlite3.connect(dbname) as conn:
        cursor = conn.cursor()
        sql = 'select * from oliveyoungreview' + whr
        res = cursor.execute(sql).fetchall()
    return res

In [32]:
def insMain(d):
    dbname = 'Oliveyoung.db'
    with sqlite3.connect(dbname) as conn:
        cursor = conn.cursor()
        sql = 'insert into oliveyoungreview (name,brand_name,review,rate,frag,frag_time,gender) values (?,?,?,?,?,?,?)'
        res = cursor.execute(sql, d)
        print('.', end='')            

In [4]:
dpath = 'C:/Users/mmqq2/Kdigital/chromedriver'
driver = webdriver.Chrome(dpath)

In [5]:
def getText_w():
    p = 1
    while True:
        url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100050003&fltDispCatNo=&prdSort=01&pageIdx={}'.format(p)
        driver.get(url)
        driver.implicitly_wait(3)
        html = driver.page_source
        htmls = bsp(html,'html.parser')
        count_p = htmls.select('p.cate_info_tx span')[0].text.strip()
        ul_list = htmls.select('ul.cate_prd_list')
        for ul in ul_list:
            prd_info = ul.select('div.prd_info > a')
            for i in range(len(prd_info)):
                prd_link = prd_info[i]['href']
                getReview(prd_link) 
        p += 1
        if p >= int(count_p)/10+1:
                break
        

In [169]:
# url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=B000000149028&dispCatNo=1000001000500030001&trackingCd=Drawer_Cat10000010005_Cat&curation&egcode&rccode&egrankcode'
# driver.get(url)
# driver.implicitly_wait(3)
# html = driver.page_source
# htmls = bsp(html,'html.parser')

# p = 2
# while True:
#     time.sleep(1)
#     driver.find_element_by_css_selector('#reviewInfo').click()
#     time.sleep(1)

#     brand = htmls.select('#moveBrandShop')[0].text
#     prd_name = htmls.select('p.prd_name')[0].text
#     gender = htmls.select('ul.loc_history > li > a')[2].text
#     review_ = driver.find_elements_by_css_selector('div.txt_inner')
#     page_btn = htmls.select('div.pageing a')
#     count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1]
#     for i in range(len(review_)):
#         review = review_[i].text
#         print('브랜드:', brand)
#         print('제품명:', prd_name)
#         print('성별:', gender)
#         print('리뷰:', review)
#         print('')
# #     print(driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).text)
#     driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()   
    
#     p += 1
#     if p >= int(count_r)/10+1:
#         break

In [28]:
def fragscore(frag):
    if frag == '아주 만족해요':
        frag = str(5)
    elif frag == '보통이에요':
        frag = str(3)
    elif frag == '다소 아쉬워요':
        frag = str(1)
    return frag

In [29]:
def frag_timescore(frag_time):
    if frag_time == '지속이 오래돼요':
        frag_time = str(5)
    elif frag_time == '보통이에요':
        frag_time = str(3)
    elif frag_time == '예상보다 짧아요':
        frag_time = str(1)
    return frag_time

In [30]:
def rate_score(rate):
    if rate == '5점만점에 5점':
        rate = str(5)
    elif rate == '5점만점에 4점':
        rate = str(4)
    elif rate == '5점만점에 3점':
        rate = str(3)
    elif rate == '5점만점에 2점':
        rate = str(2)
    elif rate == '5점만점에 1점':
        rate = str(1)
    else:
        rate = str(0)

In [31]:
def getReview(urlsrc):
    url = urlsrc
    # url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000131261&dispCatNo=1000001000500030001&trackingCd=Drawer_Cat10000010005_Cat'
    driver.get(url)
    driver.implicitly_wait(3)
    html = driver.page_source
    htmls = bsp(html,'html.parser')
    p = 2
    while True:
        time.sleep(1)
        driver.find_element_by_css_selector('#reviewInfo').click()
        time.sleep(1)
        brand = htmls.select('#moveBrandShop')[0].text
        prd_name = htmls.select('p.prd_name')[0].text
        gender_ = htmls.select('ul.loc_history > li > a')[2].text
        if gender_ == '향수':
            gender = '남성향수'
        else:
            gender = '여성향수'
        review_ = driver.find_elements_by_css_selector('div.txt_inner')
        page_btn = htmls.select('div.pageing a')
        count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1].replace(',','')
        frag_ = driver.find_elements_by_css_selector('dl.poll_type1 > dd > span.txt')
        rate_ = driver.find_elements_by_css_selector('span.review_point > span.point')
        for i in range(len(review_)):
            review = review_[i].text
            frag = frag_[3*i].text
            frag = fragscore(frag)
            frag_time = frag_[3*i+1].text
            frag_time = frag_timescore(frag_time)
            rate = rate_[i].text
            rate = rate_score(rate)
            d = (prd_name,brand,review,rate,frag,frag_time,gender)
            insMain(d)
        if int(count_r) > 10:
            driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()
            p += 1
        
        if p >= int(count_r)/10+1:
            break

In [160]:
# p = 1
# while True:
#     url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100050003&fltDispCatNo=&prdSort=01&pageIdx={}'.format(p)
#     driver.get(url)
#     driver.implicitly_wait(3)
#     html = driver.page_source
#     htmls = bsp(html,'html.parser')
#     count_p = htmls.select('p.cate_info_tx span')[0].text.strip()

#     ul_list = htmls.select('ul.cate_prd_list')
#     for ul in ul_list:
#         prd_name_ = ul.select('p.tx_name')
#         prd_info = ul.select('div.prd_info > a')
#         for i in range(len(prd_info)):
#             prd_link = prd_info[i]['href']
#             driver.get(prd_link)
#             driver.implicitly_wait(3)
#             html = driver.page_source
#             htmls = bsp(html,'html.parser')

#             p = 2
#             while True:
#                 time.sleep(1)
#                 driver.find_element_by_css_selector('#reviewInfo').click()
#                 time.sleep(1)

#                 brand = htmls.select('#moveBrandShop')[0].text
#                 prd_name = htmls.select('p.prd_name')[0].text
#                 gender = htmls.select('ul.loc_history > li > a')[2].text
#                 review_ = driver.find_elements_by_css_selector('div.txt_inner')
#                 page_btn = htmls.select('div.pageing a')
#                 count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1]
#                 for i in range(len(review_)):
#                     review = review_[i].text
#                     print('브랜드:', brand)
#                     print('제품명:', prd_name)
#                     print('성별:', gender)
#                     print('리뷰:', review)
#                     print('')
#             #     print(driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).text)
#                 driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()   

#                 p += 1
#                 if p >= int(count_r)/10+1:
#                     break
        
# #     driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()    
#     p += 1
#     if p >= int(count_p)/10+2:
#             break

브랜드: 에이딕트
제품명: 에이딕트 피버러브 네이키드 걸 솔리드 퍼퓸 30ml
성별: 여성향수
리뷰: 쇼핑백은 상자에 왜넣어주는지 모르겠지만 ㅋㅋㅋ 우선 발림성은 로션제형이다보니 부드럽게 잘 발리고 금방 스며들어요. 향은 무염버터에 새콤달콤넣고 향긋한 꽃 한방울 넣은? 그런 은은한 향이에요. 신기하게도 바르고 좀 지나면 약간 향이 달라지네요. 그래도 베이스는 똑같고 드라마틱하게 변화하는건 아니에요. 그리고 이성을 유혹하는 향이라고하는데 아직 잘 모르겠어요ㅋㅋㅋ남친한테 물어볼게요ㅋㅋㅋ
마지막으로 제가 생각하는 이 제품의 가장 큰 장점은 휴대성과 지속성입니다. 스프레이 제형과 비교했을때 정말 오래가는 편이에요. 단점은 양에비해 가격이 비싸다는 생각이 들었고(로션이다보니 금방없어짐) 향이 뭔가 호불호가 갈릴 수 있다고 생각해요. 다들 시향 후에 구매하시는것을 추천드립니다.

브랜드: 에이딕트
제품명: 에이딕트 피버러브 네이키드 걸 솔리드 퍼퓸 30ml
성별: 여성향수
리뷰: 인스타광고에서 많이보던 네이키드걸 고체향수를 올리브영에서 쉽게 구매할수 있어서 넘 좋았어요^^
휴대하기 좋은 핸드크림같은 사이즈여서 유리병향수보다 안전하게 휴대할수 있어요.
향도 좋은데 발랐을때 촉촉해서 약간의 보습력이 좋네요^^ 부담되지 않는 파우더향이어서 호불호없는 향이예요~~^^ 강력추천합니다

브랜드: 에이딕트
제품명: 에이딕트 피버러브 네이키드 걸 솔리드 퍼퓸 30ml
성별: 여성향수
리뷰: 올영에 바로 떠서 구매했어요
두번째 쓰는건데 오전에 바르면 은은하게 계속 남아 있더라구요 평소 구찌랑 딥디크 썼는데 그거 안쓰고 에이딕트만 쓰는중이에요 향은 확실히 호불호 안갈려요!

브랜드: 에이딕트
제품명: 에이딕트 피버러브 네이키드 걸 솔리드 퍼퓸 30ml
성별: 여성향수
리뷰: 솔직히 저는 잘 모르겠는데 남자가 맡았을때는 좋은거 같아요. 전철에서 옆에앉은 남자가 향수 뭐 쓰는지 물어봤어용.

브랜드: 에이딕트
제품명: 에이딕트 피버러브 네이키드 걸 솔리드 퍼퓸 30ml
성별: 여성향수
리뷰:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\mmqq2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-160-d5095c74178a>", line 23, in <module>
    time.sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mmqq2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mmqq2\Anaconda3\lib\linecache.py", line 74, in checkcache
    stat = os.stat(fullname)
OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: '<ipython-input-160-d5095c74178a>'

During handling of the above exception, another exception 

TypeError: object of type 'NoneType' has no len()

## 남성향수

In [7]:
def getText_m():
    p = 1
    while True:
        url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100050004&fltDispCatNo=&prdSort=01&pageIdx={}'.format(p)
        driver.get(url)
        driver.implicitly_wait(3)
        html = driver.page_source
        htmls = bsp(html,'html.parser')
        count_p = htmls.select('p.cate_info_tx span')[0].text.strip()
        ul_list = htmls.select('ul.cate_prd_list')
        for ul in ul_list:
            prd_info = ul.select('div.prd_info > a')
            for i in range(len(prd_info)):
                prd_link = prd_info[i]['href']
                getReview(prd_link) 
        p += 1
        if p >= int(count_p)/10+1:
                break
        

In [343]:
# getText_w()

.......................................

IndexError: list index out of range

In [ ]:
# getText_m()

In [347]:
url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000000444&dispCatNo=1000001000500030001&trackingCd=Drawer_Cat10000010005_Cat'
driver.get(url)
driver.implicitly_wait(3)
html = driver.page_source
htmls = bsp(html,'html.parser')
p = 2
while True:
    time.sleep(1)
    driver.find_element_by_css_selector('#reviewInfo').click()
    time.sleep(1)
    count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1].replace(',','')
    review_ = driver.find_elements_by_css_selector('div.txt_inner')
    frag_ = driver.find_elements_by_css_selector('dl.poll_type1 > dd > span.txt')
    rate_ = driver.find_elements_by_css_selector('span.review_point > span.point')
    for i in range(len(review_)):
        review = review_[i].text
        frag = frag_[3*i].text
    #     if frag == '아주 만족해요':
    #         frag = 5
    #     elif frag == '보통이에요':
    #         frag = 3
    #     elif frag == '다소 아쉬워요':
    #         frag = 1
        frag_time = frag_[3*i+1].text
    #     if frag_time == '지속이 오래되요':
    #         frag = 5
    #     elif frag_time == '보통이에요':
    #         frag_time = 3
    #     elif frag_time == '예상보다 짧아요':
    #         frag_time = 1
        rate = rate_[i].text
    #     if rate == '5점만점에 5점':
    #         rate = 5
    #     elif rate == '5점만점에 4점':
    #         rate = 4
    #     elif rate == '5점만점에 3점':
    #         rate = 3
    #     elif rate == '5점만점에 2점':
    #         rate = 2
    #     elif rate == '5점만점에 1점':
    #         rate = 1
    #     else:
    #         rate = 0
        print(frag,frag_time,rate)

    if int(count_r) > 10:
        driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()
        p += 1

    if p >= int(count_r)/10+1:
        break

아주 만족해요 지속이 오래돼요 
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 3점
보통이에요 보통이에요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 4점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 예상보다 짧아요 5점만점에 5점
보통이에요 보통이에요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 보통이에요 
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 5점
보통이에요 보통이에요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 4점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 예상보다 짧아요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
보통이에요 보통이에요 
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 4점
아주 만족해요 보통이에요 5점만점에 5점
아주 만족해요 지속이 오래돼요 5점만점에 5점
보통이에요 보통이에요 5점만점에 5점


IndexError: list index out of range

In [33]:
url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000000444&dispCatNo=1000001000500030001&trackingCd=Drawer_Cat100000100050003_Cat'
driver.get(url)
driver.implicitly_wait(3)
html = driver.page_source
htmls = bsp(html,'html.parser')
p = 2
while True:
    time.sleep(1)
    driver.find_element_by_css_selector('#reviewInfo').click()
    time.sleep(1)
    li_list = driver.find_elements_by_css_selector('#gdasList > li')
    count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1].replace(',','')
    frag_ = driver.find_elements_by_css_selector('div.poll_sample')
    time.sleep(1)
    for li in li_list:
        if len(frag_) == 0:
            frag = str(0)
            frag_time = str(0)
        elif len(frag_) > 0:
            frag = li.find_elements_by_css_selector('dl.poll_type1 > dd > span.txt')[0].text
            frag_time = li.find_elements_by_css_selector('dl.poll_type1 > dd > span.txt')[1]
        print(p,frag)

    if int(count_r) > 10:
        driver.find_element_by_css_selector('div.pageing > a[data-page-no="{0}"]'.format(p)).click()
        p += 1

    if p >= int(count_r)/10+1:
        break

2 아주 만족해요
2 아주 만족해요
2 아주 만족해요
2 보통이에요
2 아주 만족해요
2 아주 만족해요
2 아주 만족해요
2 보통이에요
2 아주 만족해요
2 아주 만족해요
3 아주 만족해요
3 아주 만족해요
3 아주 만족해요
3 보통이에요
3 아주 만족해요
3 아주 만족해요
3 아주 만족해요
3 아주 만족해요
3 아주 만족해요
3 아주 만족해요
4 보통이에요
4 아주 만족해요
4 아주 만족해요


IndexError: list index out of range

In [22]:
url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000000444&dispCatNo=1000001000500030001&trackingCd=Drawer_Cat100000100050003_Cat'
driver.get(url)
driver.implicitly_wait(3)
html = driver.page_source
htmls = bsp(html,'html.parser')

time.sleep(1)
driver.find_element_by_css_selector('#reviewInfo').click()
time.sleep(1)
li_list = driver.find_elements_by_css_selector('#gdasList > li')
count_r = driver.find_elements_by_css_selector('p.total')[0].text.split(' ')[1].replace(',','')
frag_ = driver.find_elements_by_css_selector('div.poll_sample')
time.sleep(1)

for li in li_list:
    frag = li.find_elements_by_css_selector('dl.poll_type1 > dd > span.txt')[1].text
    print(frag)

지속이 오래돼요
지속이 오래돼요
지속이 오래돼요
보통이에요
보통이에요
보통이에요
예상보다 짧아요
보통이에요
지속이 오래돼요
보통이에요
